In [1]:
try:
    import google.colab  # noqa: F401

    %pip install -q dataeval
except Exception:
    pass

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import models

# Drift
from dataeval.detectors.drift import DriftCVM, DriftKS, DriftMMD
from dataeval.metrics.bias import label_parity
from dataeval.utils.data import Embeddings, Metadata
from dataeval.utils.data.datasets import VOCDetectionTorch

# Set a random seed
rng = np.random.default_rng(213)

# Set default torch device for notebook
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)

In [3]:
# Define the embedding network
class EmbeddingNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Load in pretrained resnet18 model
        self.model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        # Add an additional fully connected layer with an embedding dimension of 128
        self.model.fc = nn.Linear(self.model.fc.in_features, 128)

    def forward(self, x):
        """Run input data through the model"""

        return self.model(x)

In [4]:
embedding_net = EmbeddingNet()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/dataeval/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0% 0.00/44.7M [00:00<?, ?B/s]

 40% 17.9M/44.7M [00:00<00:00, 187MB/s]

 91% 40.8M/44.7M [00:00<00:00, 217MB/s]

100% 44.7M/44.7M [00:00<00:00, 214MB/s]

In [5]:
# Define pretrained model transformations
transforms = models.ResNet18_Weights.DEFAULT.transforms()

# Load the training dataset
train_ds = VOCDetectionTorch("./data", year="2011", image_set="train", download=False, transforms=transforms)
# Load the "operational" dataset
operational_ds = VOCDetectionTorch("./data", year="2011", image_set="val", download=False, transforms=transforms)

print(train_ds)
print(operational_ds)

VOCDetectionTorch Dataset
-------------------------
    Year: 2011
    Transforms: [ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)]
    Image_set: train
    Metadata: {'id': 'VOCDetectionTorch_train', 'index2label': {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle', 5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow', 10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person', 15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}, 'split': 'train'}
    Path: /dataeval/docs/source/tutorials/data/VOC2011
    Size: 5717
VOCDetectionTorch Dataset
-------------------------
    Year: 2011
    Transforms: [ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)]
    Image_set: val
    Metadata: {'id': 'VOCDetec

In [6]:
# Create training batches and targets
train_embs = Embeddings(train_ds, batch_size=64, model=embedding_net).to_tensor()

# Create operational batches and targets
operational_embs = Embeddings(operational_ds, batch_size=64, model=embedding_net).to_tensor()

In [7]:
print(train_embs.shape)
print(operational_embs.shape)

torch.Size([5717, 128])
torch.Size([5823, 128])


In [8]:
# A type alias for all of the drift detectors
DriftDetector = DriftMMD | DriftCVM | DriftKS

# Create a mapping for the detectors to iterate over
detectors: dict[str, DriftDetector] = {
    "MMD": DriftMMD(train_embs),
    "CVM": DriftCVM(train_embs),
    "KS": DriftKS(train_embs),
}

In [9]:
# Iterate and print the name of the detector class and its boolean drift prediction
for name, detector in detectors.items():
    print(f"{name} detected drift? {detector.predict(operational_embs).drifted}")

MMD detected drift? False
CVM detected drift? False


KS detected drift? False


In [10]:
# Creates a normal distribution around the operational embeddings
noisy_embs = torch.normal(mean=operational_embs)

In [11]:
# Iterate and print the name of the detector class and its boolean drift prediction
for name, detector in detectors.items():
    print(f"{name} detected drift? {detector.predict(noisy_embs).drifted}")

MMD detected drift? True


CVM detected drift? True


KS detected drift? True


In [12]:
# The VOC dataset has 20 classes
label_parity(Metadata(train_ds).targets.labels, Metadata(operational_ds).targets.labels, num_classes=20).p_value

np.float64(0.949856067521638)